In [1]:
# 필요한 모듈 import

import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [2]:
# 구글 드라이브 연동

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
code=pd.read_csv("/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/0. 원본데이터/행정동.csv")
code=code[['ADSTRD_CD','SIGNGU_CD']]
code=code.rename(columns={'ADSTRD_CD':'행정동코드', 'SIGNGU_CD':'시군구코드'})
code['행정동코드']=code['행정동코드']//100
code['행정동코드']=code['행정동코드'].astype('int64')
code

,행정동코드,시군구코드
0,11110515,11110
1,11110530,11110
2,11110540,11110
3,11110550,11110
4,11110560,11110
...,...,...
420,11740650,11740
421,11740660,11740
422,11740685,11740
423,11740690,11740


# 직장인구 데이터

- 다운로드: https://data.seoul.go.kr/dataList/OA-15569/S/1/datasetView.do

In [4]:
company=pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/0. 원본데이터/직장인구/서울시 상권분석서비스(상권-직장인구).csv', header=0, encoding = 'cp949')
company=company[['기준_년월_코드', '기준_분기_코드', '상권_코드', '상권_코드_명', '총_직장_인구_수', '남성_직장_인구_수', '여성_직장_인구_수']]
company

,기준_년월_코드,기준_분기_코드,상권_코드,상권_코드_명,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수
0,2023,1,3001496,강남 마이스 관광특구,15904,8904,7000
1,2023,1,3001495,잠실 관광특구,25240,14855,10385
2,2023,1,3001494,종로?청계 관광특구,32892,17118,15774
3,2023,1,3001493,동대문패션타운 관광특구,5916,2900,3016
4,2023,1,3001492,명동 남대문 북창동 다동 무교동 관광특구,104830,53674,51156
...,...,...,...,...,...,...,...
39362,2017,2,2110005,세검정,445,159,286
39363,2017,2,2110004,대신고등학교,41,14,27
39364,2017,2,2110003,세검정초등학교,144,79,65
39365,2017,2,2110002,독립문역 1번,583,152,431


In [5]:
new_1=pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/0. 원본데이터/신상권코드-행정동코드.csv',encoding='CP949')
new_1=new_1[["상권_코드","행정동_코드"]]
new_2=pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/0. 원본데이터/신신상권코드-행정동코드.csv',encoding='CP949')
new_2=new_2[["상권_코드","행정동_코드"]]

new=pd.concat([new_1, new_2], axis = 0)
new

,상권_코드,행정동_코드
0,2110753,11545710
1,2110427,11350611
2,2110755,11545680
3,2110739,11545690
4,2111017,11710562
...,...,...
1645,3001492,11140520
1646,3001493,11140590
1647,3001494,11110615
1648,3001495,11710566


In [6]:
company=pd.merge(company,new,on=['상권_코드'],how='left')
company=company.drop(['상권_코드_명','상권_코드'],axis=1)
company.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 39367 entries, 0 to 39366
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   기준_년월_코드    39367 non-null  int64
 1   기준_분기_코드    39367 non-null  int64
 2   총_직장_인구_수   39367 non-null  int64
 3   남성_직장_인구_수  39367 non-null  int64
 4   여성_직장_인구_수  39367 non-null  int64
 5   행정동_코드      39367 non-null  int64
dtypes: int64(6)
memory usage: 2.1 MB


In [7]:
company=company.rename(columns={'행정동_코드':'행정동코드', '기준_년월_코드':'년도', '기준_분기_코드':'분기'})
company=company.groupby(['년도','분기','행정동코드']).agg({'총_직장_인구_수':np.sum,'남성_직장_인구_수':np.sum,'여성_직장_인구_수':np.sum}).reset_index()
company

,년도,분기,행정동코드,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수
0,2017,2,11110515,1299,749,550
1,2017,2,11110530,93104,53828,39276
2,2017,2,11110540,1159,497,662
3,2017,2,11110550,964,387,577
4,2017,2,11110560,1513,685,828
...,...,...,...,...,...,...
9613,2023,1,11740640,11836,8186,3650
9614,2023,1,11740650,1215,693,522
9615,2023,1,11740660,3709,2234,1475
9616,2023,1,11740685,8820,4395,4425


In [8]:
company=company.sort_values(['년도', '분기', '행정동코드'])
company=company.reset_index(drop=True)
company=company.drop_duplicates()
company

,년도,분기,행정동코드,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수
0,2017,2,11110515,1299,749,550
1,2017,2,11110530,93104,53828,39276
2,2017,2,11110540,1159,497,662
3,2017,2,11110550,964,387,577
4,2017,2,11110560,1513,685,828
...,...,...,...,...,...,...
9613,2023,1,11740640,11836,8186,3650
9614,2023,1,11740650,1215,693,522
9615,2023,1,11740660,3709,2234,1475
9616,2023,1,11740685,8820,4395,4425


In [10]:
sales=pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/1. 전처리/sales.csv')
sales_temp=sales[['년도', '분기', '행정동코드']]
sales_temp=sales_temp.drop_duplicates()

temp=pd.merge(sales_temp, company, on=['년도', '분기', '행정동코드'], how='outer')
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9864 entries, 0 to 9863
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년도          9864 non-null   int64  
 1   분기          9864 non-null   int64  
 2   행정동코드       9864 non-null   int64  
 3   총_직장_인구_수   9618 non-null   float64
 4   남성_직장_인구_수  9618 non-null   float64
 5   여성_직장_인구_수  9618 non-null   float64
dtypes: float64(3), int64(3)
memory usage: 539.4 KB


In [11]:
temp.sort_values(by=['년도', '분기'], inplace=True)
temp['순서'] = temp.groupby(['년도', '분기']).ngroup() + 1

temp

,년도,분기,행정동코드,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,순서
3395,2017,2,11110515,1299.0,749.0,550.0,1
3396,2017,2,11110530,93104.0,53828.0,39276.0,1
3397,2017,2,11110540,1159.0,497.0,662.0,1
3398,2017,2,11110550,964.0,387.0,577.0,1
3399,2017,2,11110560,1513.0,685.0,828.0,1
...,...,...,...,...,...,...,...
9859,2023,1,11740640,11836.0,8186.0,3650.0,24
9860,2023,1,11740650,1215.0,693.0,522.0,24
9861,2023,1,11740660,3709.0,2234.0,1475.0,24
9862,2023,1,11740685,8820.0,4395.0,4425.0,24


In [12]:
columns_to_fill = ['총_직장_인구_수', '남성_직장_인구_수', '여성_직장_인구_수']

for column in columns_to_fill:
    for index, row in temp[temp[column].isnull()].iterrows():
        candidate_rows = temp[
            (temp['행정동코드'] == row['행정동코드']) &
            ((temp['순서'] == row['순서'] - 1) | (temp['순서'] == row['순서'] + 1))
        ]

        if not candidate_rows.empty:
            avg_value = candidate_rows[column].mean()
            temp.at[index, column] = avg_value

temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9864 entries, 3395 to 9863
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년도          9864 non-null   int64  
 1   분기          9864 non-null   int64  
 2   행정동코드       9864 non-null   int64  
 3   총_직장_인구_수   9618 non-null   float64
 4   남성_직장_인구_수  9618 non-null   float64
 5   여성_직장_인구_수  9618 non-null   float64
 6   순서          9864 non-null   int64  
dtypes: float64(3), int64(4)
memory usage: 874.5 KB


In [13]:
temp['행정동코드']=temp['행정동코드'].astype('int64')
temp=pd.merge(temp,code,on=['행정동코드'], how='left')
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9864 entries, 0 to 9863
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년도          9864 non-null   int64  
 1   분기          9864 non-null   int64  
 2   행정동코드       9864 non-null   int64  
 3   총_직장_인구_수   9618 non-null   float64
 4   남성_직장_인구_수  9618 non-null   float64
 5   여성_직장_인구_수  9618 non-null   float64
 6   순서          9864 non-null   int64  
 7   시군구코드       9720 non-null   float64
dtypes: float64(4), int64(4)
memory usage: 693.6 KB


In [14]:
fill_mean_func = lambda g: g.fillna(g.mean())
temp=temp.groupby(['년도','분기','시군구코드']).apply(fill_mean_func)
temp.info()

<ipython-input-14-1e7f7c502bc4>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  temp=temp.groupby(['년도','분기','시군구코드']).apply(fill_mean_func)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9720 entries, 0 to 9863
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년도          9720 non-null   int64  
 1   분기          9720 non-null   int64  
 2   행정동코드       9720 non-null   int64  
 3   총_직장_인구_수   9720 non-null   float64
 4   남성_직장_인구_수  9720 non-null   float64
 5   여성_직장_인구_수  9720 non-null   float64
 6   순서          9720 non-null   int64  
 7   시군구코드       9720 non-null   float64
dtypes: float64(4), int64(4)
memory usage: 941.5 KB


In [15]:
company=pd.merge(sales, temp, on=['년도', '분기', '행정동코드'], how='left')
company

,년도,분기,행정동코드,표준산업분류코드,경쟁업체개수,프랜차이즈비율,grade,result,sales,infra,franch,pop,fin,분기당_매출_금액,분기당_매출_건수,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,점포수,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,순서,시군구코드
0,2020,1,11110515,47121,1.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,3.409683e+09,168261.0,520072278.0,476737496.0,454100540.0,497622093.0,539107397.0,533638822.0,388404258.0,25021.0,24228.0,22816.0,23552.0,25232.0,26351.0,21061.0,6.0,1646.0,1023.0,623.0,12,11110.0
1,2020,1,11110515,47122,11.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,6.808622e+08,95872.0,87547259.0,92392709.0,91715951.0,85431530.0,103510713.0,110703586.0,109560447.0,12007.0,12675.0,12992.0,12942.0,13812.0,15917.0,15527.0,9.0,1646.0,1023.0,623.0,12,11110.0
2,2020,1,11110515,47214,24.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,2.540552e+08,15810.0,31750095.0,39373123.0,36153051.0,36434690.0,53526417.0,41149325.0,15668512.0,2178.0,2459.0,2178.0,2328.0,2779.0,2780.0,1108.0,6.0,1646.0,1023.0,623.0,12,11110.0
3,2020,1,11110515,47219,10.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,1.278657e+09,35217.4,169709975.2,145993455.2,215581422.1,268772815.0,250489180.3,141215681.7,86894377.0,5715.3,4902.9,4528.6,5427.1,6233.1,5018.3,3392.1,21.5,1646.0,1023.0,623.0,12,11110.0
4,2020,1,11110515,47416,31.500000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,2.596228e+08,2510.0,17179799.0,19191157.0,30019657.0,37653338.0,29524120.0,52551635.0,73503106.0,89.0,162.0,370.0,352.0,285.0,714.0,538.0,26.0,1646.0,1023.0,623.0,12,11110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79036,2021,4,11740700,91136,11.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,2.941996e+08,7169.0,26951709.0,45763609.0,26331440.0,43324892.0,34529151.0,64672701.0,52626083.0,704.0,905.0,891.0,806.0,999.0,1628.0,1236.0,11.0,1807.0,1048.0,759.0,19,11740.0
79037,2021,4,11740700,91223,16.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,1.195457e+08,1316.0,21697068.0,6605387.0,23355840.0,13352504.0,23790394.0,19549380.0,11195089.0,160.0,117.0,184.0,113.0,326.0,275.0,141.0,24.0,1807.0,1048.0,759.0,19,11740.0
79038,2021,4,11740700,96112,63.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,3.271878e+08,11441.0,45385363.0,29317905.0,34170843.0,45287742.0,62706433.0,71777182.0,38542355.0,1464.0,990.0,1299.0,1658.0,2075.0,2410.0,1545.0,79.0,1807.0,1048.0,759.0,19,11740.0
79039,2021,4,11740700,96113,18.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,1.144000e+07,89.0,1240000.0,0.0,800000.0,0.0,9400000.0,0.0,0.0,20.0,0.0,20.0,0.0,49.0,0.0,0.0,5.0,1807.0,1048.0,759.0,19,11740.0


In [16]:
company=company.dropna()
company=company.drop(['시군구코드', '순서'],axis=1)
company

,년도,분기,행정동코드,표준산업분류코드,경쟁업체개수,프랜차이즈비율,grade,result,sales,infra,franch,pop,fin,분기당_매출_금액,분기당_매출_건수,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,점포수,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수
0,2020,1,11110515,47121,1.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,3.409683e+09,168261.0,520072278.0,476737496.0,454100540.0,497622093.0,539107397.0,533638822.0,388404258.0,25021.0,24228.0,22816.0,23552.0,25232.0,26351.0,21061.0,6.0,1646.0,1023.0,623.0
1,2020,1,11110515,47122,11.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,6.808622e+08,95872.0,87547259.0,92392709.0,91715951.0,85431530.0,103510713.0,110703586.0,109560447.0,12007.0,12675.0,12992.0,12942.0,13812.0,15917.0,15527.0,9.0,1646.0,1023.0,623.0
2,2020,1,11110515,47214,24.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,2.540552e+08,15810.0,31750095.0,39373123.0,36153051.0,36434690.0,53526417.0,41149325.0,15668512.0,2178.0,2459.0,2178.0,2328.0,2779.0,2780.0,1108.0,6.0,1646.0,1023.0,623.0
3,2020,1,11110515,47219,10.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,1.278657e+09,35217.4,169709975.2,145993455.2,215581422.1,268772815.0,250489180.3,141215681.7,86894377.0,5715.3,4902.9,4528.6,5427.1,6233.1,5018.3,3392.1,21.5,1646.0,1023.0,623.0
4,2020,1,11110515,47416,31.500000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,2.596228e+08,2510.0,17179799.0,19191157.0,30019657.0,37653338.0,29524120.0,52551635.0,73503106.0,89.0,162.0,370.0,352.0,285.0,714.0,538.0,26.0,1646.0,1023.0,623.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79036,2021,4,11740700,91136,11.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,2.941996e+08,7169.0,26951709.0,45763609.0,26331440.0,43324892.0,34529151.0,64672701.0,52626083.0,704.0,905.0,891.0,806.0,999.0,1628.0,1236.0,11.0,1807.0,1048.0,759.0
79037,2021,4,11740700,91223,16.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,1.195457e+08,1316.0,21697068.0,6605387.0,23355840.0,13352504.0,23790394.0,19549380.0,11195089.0,160.0,117.0,184.0,113.0,326.0,275.0,141.0,24.0,1807.0,1048.0,759.0
79038,2021,4,11740700,96112,63.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,3.271878e+08,11441.0,45385363.0,29317905.0,34170843.0,45287742.0,62706433.0,71777182.0,38542355.0,1464.0,990.0,1299.0,1658.0,2075.0,2410.0,1545.0,79.0,1807.0,1048.0,759.0
79039,2021,4,11740700,96113,18.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,1.144000e+07,89.0,1240000.0,0.0,800000.0,0.0,9400000.0,0.0,0.0,20.0,0.0,20.0,0.0,49.0,0.0,0.0,5.0,1807.0,1048.0,759.0


In [17]:
company.to_csv('./company.csv', index=False)

# 소득 소비 데이터

- 다운로드: https://data.seoul.go.kr/dataList/OA-21278/S/1/datasetView.do#

In [18]:
income = pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/0. 원본데이터/소득-소비/서울시 상권분석서비스(상권-소득소비).csv', header=0, encoding='CP949')
income = income[['기준_년_코드', '기준_분기_코드', '상권_코드', '상권_코드_명', '월_평균_소득_금액', '소득_구간_코드', '지출_총금액']]
income

,기준_년_코드,기준_분기_코드,상권_코드,상권_코드_명,월_평균_소득_금액,소득_구간_코드,지출_총금액
0,2023,1,2110001,이북5도청사,NaN,NaN,5.204276e+08
1,2023,1,2110002,독립문역 1번,NaN,NaN,2.324652e+08
2,2023,1,2110003,세검정초등학교,NaN,NaN,1.349699e+09
3,2023,1,2110004,대신고등학교,NaN,NaN,7.127728e+08
4,2023,1,2110005,세검정,NaN,NaN,8.847016e+08
...,...,...,...,...,...,...,...
40675,2017,1,1001492,명동 남대문 북창동 다동 무교동 관광특구,3374268.0,7.0,3.927186e+08
40676,2017,1,1001493,동대문패션타운 관광특구,3703242.0,7.0,2.575916e+09
40677,2017,1,1001494,종로?청계 관광특구,2350877.0,5.0,2.640514e+09
40678,2017,1,1001495,잠실 관광특구,4738833.0,8.0,7.600040e+09


In [19]:
new=pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/0. 원본데이터/신상권코드-행정동코드.csv',encoding='CP949')
new=new[["상권_코드","행정동_코드"]]
new

,상권_코드,행정동_코드
0,2110753,11545710
1,2110427,11350611
2,2110755,11545680
3,2110739,11545690
4,2111017,11710562
...,...,...
1666,1001493,11140590
1667,1001494,11110615
1668,1001495,11710566
1669,1001496,11680580


In [20]:
income=pd.merge(income, new,on=['상권_코드'],how='left')
income=income.drop(['상권_코드_명','상권_코드'],axis=1)
income

,기준_년_코드,기준_분기_코드,월_평균_소득_금액,소득_구간_코드,지출_총금액,행정동_코드
0,2023,1,NaN,NaN,5.204276e+08,11110560.0
1,2023,1,NaN,NaN,2.324652e+08,11110570.0
2,2023,1,NaN,NaN,1.349699e+09,11110550.0
3,2023,1,NaN,NaN,7.127728e+08,11110570.0
4,2023,1,NaN,NaN,8.847016e+08,11110550.0
...,...,...,...,...,...,...
40675,2017,1,3374268.0,7.0,3.927186e+08,11140520.0
40676,2017,1,3703242.0,7.0,2.575916e+09,11140590.0
40677,2017,1,2350877.0,5.0,2.640514e+09,11110615.0
40678,2017,1,4738833.0,8.0,7.600040e+09,11710566.0


In [21]:
income=income[income['기준_년_코드']<2023]
income=income[income['월_평균_소득_금액'].notnull()]
income=income[income['지출_총금액'].notnull()]
income

,기준_년_코드,기준_분기_코드,월_평균_소득_금액,소득_구간_코드,지출_총금액,행정동_코드
1598,2022,4,4305934.0,8.0,5.204276e+08,11110560.0
1599,2022,4,3463885.0,7.0,2.324652e+08,11110570.0
1600,2022,4,3548020.0,7.0,1.349699e+09,11110550.0
1601,2022,4,2644953.0,6.0,7.127728e+08,11110570.0
1602,2022,4,3703145.0,7.0,8.847016e+08,11110550.0
...,...,...,...,...,...,...
40674,2017,1,2940759.0,6.0,6.949260e+09,11170650.0
40675,2017,1,3374268.0,7.0,3.927186e+08,11140520.0
40676,2017,1,3703242.0,7.0,2.575916e+09,11140590.0
40677,2017,1,2350877.0,5.0,2.640514e+09,11110615.0


In [22]:
income.info()

#결측치: 월_평균_소득_금액은 10개 상권에 대해서는 아예 측정이 안됨. -> 버림 / 2023년은 전체적으로 측정이 안됨 -> 근데 23년 데이터 안 쓸 것 같으니까 버림
#지출_총금액 역시 33개 상권에 대해서는 측정이 안 된 것으로 보임. -> 버림

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38069 entries, 1598 to 40678
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   기준_년_코드     38069 non-null  int64  
 1   기준_분기_코드    38069 non-null  int64  
 2   월_평균_소득_금액  38069 non-null  float64
 3   소득_구간_코드    38069 non-null  float64
 4   지출_총금액      38069 non-null  float64
 5   행정동_코드      38069 non-null  float64
dtypes: float64(4), int64(2)
memory usage: 2.0 MB


In [23]:
income=income.groupby(['기준_년_코드','기준_분기_코드','행정동_코드']).agg({'월_평균_소득_금액':np.mean,'소득_구간_코드':np.mean,'지출_총금액':np.sum}).reset_index()
income

,기준_년_코드,기준_분기_코드,행정동_코드,월_평균_소득_금액,소득_구간_코드,지출_총금액
0,2017,1,11110515.0,4.246631e+06,8.000000,8.050049e+09
1,2017,1,11110530.0,3.994140e+06,7.500000,1.053332e+10
2,2017,1,11110540.0,3.732510e+06,7.000000,1.659309e+09
3,2017,1,11110550.0,3.734505e+06,7.200000,7.217523e+09
4,2017,1,11110560.0,4.902522e+06,8.333333,8.933697e+09
...,...,...,...,...,...,...
9532,2022,4,11740640.0,2.966367e+06,6.400000,7.490410e+09
9533,2022,4,11740650.0,2.606738e+06,5.666667,8.269094e+09
9534,2022,4,11740660.0,2.852998e+06,6.200000,1.044076e+10
9535,2022,4,11740685.0,2.925398e+06,6.375000,1.815481e+10


In [24]:
income=income.rename(columns={'행정동_코드':'행정동코드', '기준_분기_코드':'분기', '기준_년_코드':'년도'})
income=income[['년도', '분기', '행정동코드', '월_평균_소득_금액', '소득_구간_코드', '지출_총금액']]
income=income.sort_values(['년도', '분기', '행정동코드'])
income=income.reset_index(drop=True)
income=income.drop_duplicates()
income

,년도,분기,행정동코드,월_평균_소득_금액,소득_구간_코드,지출_총금액
0,2017,1,11110515.0,4.246631e+06,8.000000,8.050049e+09
1,2017,1,11110530.0,3.994140e+06,7.500000,1.053332e+10
2,2017,1,11110540.0,3.732510e+06,7.000000,1.659309e+09
3,2017,1,11110550.0,3.734505e+06,7.200000,7.217523e+09
4,2017,1,11110560.0,4.902522e+06,8.333333,8.933697e+09
...,...,...,...,...,...,...
9532,2022,4,11740640.0,2.966367e+06,6.400000,7.490410e+09
9533,2022,4,11740650.0,2.606738e+06,5.666667,8.269094e+09
9534,2022,4,11740660.0,2.852998e+06,6.200000,1.044076e+10
9535,2022,4,11740685.0,2.925398e+06,6.375000,1.815481e+10


In [28]:
company=pd.read_csv('/content/drive/Shareddrives/서울시 빅데이터(2022-2)/논문/1. 전처리/company.csv')
company_temp=company[['년도', '분기', '행정동코드']]
company_temp=company_temp.drop_duplicates()

temp=pd.merge(company_temp, income, on=['년도', '분기', '행정동코드'], how='outer')
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9807 entries, 0 to 9806
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년도          9807 non-null   int64  
 1   분기          9807 non-null   int64  
 2   행정동코드       9807 non-null   float64
 3   월_평균_소득_금액  9537 non-null   float64
 4   소득_구간_코드    9537 non-null   float64
 5   지출_총금액      9537 non-null   float64
dtypes: float64(4), int64(2)
memory usage: 536.3 KB


In [29]:
temp.sort_values(by=['년도', '분기'], inplace=True)
temp['순서'] = temp.groupby(['년도', '분기']).ngroup() + 1

temp

,년도,분기,행정동코드,월_평균_소득_금액,소득_구간_코드,지출_총금액,순서
3395,2017,1,11110515.0,4.246631e+06,8.000000,8.050049e+09,1
3396,2017,1,11110530.0,3.994140e+06,7.500000,1.053332e+10,1
3397,2017,1,11110540.0,3.732510e+06,7.000000,1.659309e+09,1
3398,2017,1,11110550.0,3.734505e+06,7.200000,7.217523e+09,1
3399,2017,1,11110560.0,4.902522e+06,8.333333,8.933697e+09,1
...,...,...,...,...,...,...,...
9802,2022,4,11740640.0,2.966367e+06,6.400000,7.490410e+09,24
9803,2022,4,11740650.0,2.606738e+06,5.666667,8.269094e+09,24
9804,2022,4,11740660.0,2.852998e+06,6.200000,1.044076e+10,24
9805,2022,4,11740685.0,2.925398e+06,6.375000,1.815481e+10,24


In [30]:
columns_to_fill = ['월_평균_소득_금액', '소득_구간_코드', '지출_총금액']

for column in columns_to_fill:
    for index, row in temp[temp[column].isnull()].iterrows():
        candidate_rows = temp[
            (temp['행정동코드'] == row['행정동코드']) &
            ((temp['순서'] == row['순서'] - 1) | (temp['순서'] == row['순서'] + 1))
        ]

        if not candidate_rows.empty:
            avg_value = candidate_rows[column].mean()
            temp.at[index, column] = avg_value

temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9807 entries, 3395 to 9806
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년도          9807 non-null   int64  
 1   분기          9807 non-null   int64  
 2   행정동코드       9807 non-null   float64
 3   월_평균_소득_금액  9537 non-null   float64
 4   소득_구간_코드    9537 non-null   float64
 5   지출_총금액      9537 non-null   float64
 6   순서          9807 non-null   int64  
dtypes: float64(4), int64(3)
memory usage: 871.0 KB


In [31]:
temp['행정동코드']=temp['행정동코드'].astype('int64')
temp=pd.merge(temp,code,on=['행정동코드'], how='left')
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9807 entries, 0 to 9806
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년도          9807 non-null   int64  
 1   분기          9807 non-null   int64  
 2   행정동코드       9807 non-null   int64  
 3   월_평균_소득_금액  9537 non-null   float64
 4   소득_구간_코드    9537 non-null   float64
 5   지출_총금액      9537 non-null   float64
 6   순서          9807 non-null   int64  
 7   시군구코드       9663 non-null   float64
dtypes: float64(4), int64(4)
memory usage: 689.6 KB


In [33]:
fill_mean_func = lambda g: g.fillna(g.mean())
temp=temp.groupby(['년도','분기','시군구코드']).apply(fill_mean_func)
temp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9663 entries, 0 to 9806
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   년도          9663 non-null   int64  
 1   분기          9663 non-null   int64  
 2   행정동코드       9663 non-null   int64  
 3   월_평균_소득_금액  9663 non-null   float64
 4   소득_구간_코드    9663 non-null   float64
 5   지출_총금액      9663 non-null   float64
 6   순서          9663 non-null   int64  
 7   시군구코드       9663 non-null   float64
dtypes: float64(4), int64(4)
memory usage: 937.5 KB


<ipython-input-33-1e7f7c502bc4>:2: FutureWarning: Not prepending group keys to the result index of transform-like apply. In the future, the group keys will be included in the index, regardless of whether the applied function returns a like-indexed object.
To preserve the previous behavior, use

	>>> .groupby(..., group_keys=False)

To adopt the future behavior and silence this warning, use 

	>>> .groupby(..., group_keys=True)
  temp=temp.groupby(['년도','분기','시군구코드']).apply(fill_mean_func)


In [34]:
income=pd.merge(company, temp, on=['년도', '분기', '행정동코드'], how='left')
income

,년도,분기,행정동코드,표준산업분류코드,경쟁업체개수,프랜차이즈비율,grade,result,sales,infra,franch,pop,fin,분기당_매출_금액,분기당_매출_건수,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,점포수,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,월_평균_소득_금액,소득_구간_코드,지출_총금액,순서,시군구코드
0,2020,1,11110515,47121,1.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,3.409683e+09,168261.0,520072278.0,476737496.0,454100540.0,497622093.0,539107397.0,533638822.0,388404258.0,25021.0,24228.0,22816.0,23552.0,25232.0,26351.0,21061.0,6.0,1646.0,1023.0,623.0,4110587.0,7.8,3.932929e+09,13,11110.0
1,2020,1,11110515,47122,11.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,6.808622e+08,95872.0,87547259.0,92392709.0,91715951.0,85431530.0,103510713.0,110703586.0,109560447.0,12007.0,12675.0,12992.0,12942.0,13812.0,15917.0,15527.0,9.0,1646.0,1023.0,623.0,4110587.0,7.8,3.932929e+09,13,11110.0
2,2020,1,11110515,47214,24.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,2.540552e+08,15810.0,31750095.0,39373123.0,36153051.0,36434690.0,53526417.0,41149325.0,15668512.0,2178.0,2459.0,2178.0,2328.0,2779.0,2780.0,1108.0,6.0,1646.0,1023.0,623.0,4110587.0,7.8,3.932929e+09,13,11110.0
3,2020,1,11110515,47219,10.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,1.278657e+09,35217.4,169709975.2,145993455.2,215581422.1,268772815.0,250489180.3,141215681.7,86894377.0,5715.3,4902.9,4528.6,5427.1,6233.1,5018.3,3392.1,21.5,1646.0,1023.0,623.0,4110587.0,7.8,3.932929e+09,13,11110.0
4,2020,1,11110515,47416,31.500000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,2.596228e+08,2510.0,17179799.0,19191157.0,30019657.0,37653338.0,29524120.0,52551635.0,73503106.0,89.0,162.0,370.0,352.0,285.0,714.0,538.0,26.0,1646.0,1023.0,623.0,4110587.0,7.8,3.932929e+09,13,11110.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79036,2021,4,11740700,91136,11.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,2.941996e+08,7169.0,26951709.0,45763609.0,26331440.0,43324892.0,34529151.0,64672701.0,52626083.0,704.0,905.0,891.0,806.0,999.0,1628.0,1236.0,11.0,1807.0,1048.0,759.0,2758606.6,6.0,6.515563e+09,20,11740.0
79037,2021,4,11740700,91223,16.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,1.195457e+08,1316.0,21697068.0,6605387.0,23355840.0,13352504.0,23790394.0,19549380.0,11195089.0,160.0,117.0,184.0,113.0,326.0,275.0,141.0,24.0,1807.0,1048.0,759.0,2758606.6,6.0,6.515563e+09,20,11740.0
79038,2021,4,11740700,96112,63.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,3.271878e+08,11441.0,45385363.0,29317905.0,34170843.0,45287742.0,62706433.0,71777182.0,38542355.0,1464.0,990.0,1299.0,1658.0,2075.0,2410.0,1545.0,79.0,1807.0,1048.0,759.0,2758606.6,6.0,6.515563e+09,20,11740.0
79039,2021,4,11740700,96113,18.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,1.144000e+07,89.0,1240000.0,0.0,800000.0,0.0,9400000.0,0.0,0.0,20.0,0.0,20.0,0.0,49.0,0.0,0.0,5.0,1807.0,1048.0,759.0,2758606.6,6.0,6.515563e+09,20,11740.0


In [35]:
income=income.dropna()
income=income.drop(['시군구코드', '순서'],axis=1)
income

,년도,분기,행정동코드,표준산업분류코드,경쟁업체개수,프랜차이즈비율,grade,result,sales,infra,franch,pop,fin,분기당_매출_금액,분기당_매출_건수,월요일_매출_금액,화요일_매출_금액,수요일_매출_금액,목요일_매출_금액,금요일_매출_금액,토요일_매출_금액,일요일_매출_금액,월요일_매출_건수,화요일_매출_건수,수요일_매출_건수,목요일_매출_건수,금요일_매출_건수,토요일_매출_건수,일요일_매출_건수,점포수,총_직장_인구_수,남성_직장_인구_수,여성_직장_인구_수,월_평균_소득_금액,소득_구간_코드,지출_총금액
0,2020,1,11110515,47121,1.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,3.409683e+09,168261.0,520072278.0,476737496.0,454100540.0,497622093.0,539107397.0,533638822.0,388404258.0,25021.0,24228.0,22816.0,23552.0,25232.0,26351.0,21061.0,6.0,1646.0,1023.0,623.0,4110587.0,7.8,3.932929e+09
1,2020,1,11110515,47122,11.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,6.808622e+08,95872.0,87547259.0,92392709.0,91715951.0,85431530.0,103510713.0,110703586.0,109560447.0,12007.0,12675.0,12992.0,12942.0,13812.0,15917.0,15527.0,9.0,1646.0,1023.0,623.0,4110587.0,7.8,3.932929e+09
2,2020,1,11110515,47214,24.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,2.540552e+08,15810.0,31750095.0,39373123.0,36153051.0,36434690.0,53526417.0,41149325.0,15668512.0,2178.0,2459.0,2178.0,2328.0,2779.0,2780.0,1108.0,6.0,1646.0,1023.0,623.0,4110587.0,7.8,3.932929e+09
3,2020,1,11110515,47219,10.000000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,1.278657e+09,35217.4,169709975.2,145993455.2,215581422.1,268772815.0,250489180.3,141215681.7,86894377.0,5715.3,4902.9,4528.6,5427.1,6233.1,5018.3,3392.1,21.5,1646.0,1023.0,623.0,4110587.0,7.8,3.932929e+09
4,2020,1,11110515,47416,31.500000,0.028311,19.666667,8.0,2.666667,6.00,1.00,6.00,14.333333,2.596228e+08,2510.0,17179799.0,19191157.0,30019657.0,37653338.0,29524120.0,52551635.0,73503106.0,89.0,162.0,370.0,352.0,285.0,714.0,538.0,26.0,1646.0,1023.0,623.0,4110587.0,7.8,3.932929e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79036,2021,4,11740700,91136,11.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,2.941996e+08,7169.0,26951709.0,45763609.0,26331440.0,43324892.0,34529151.0,64672701.0,52626083.0,704.0,905.0,891.0,806.0,999.0,1628.0,1236.0,11.0,1807.0,1048.0,759.0,2758606.6,6.0,6.515563e+09
79037,2021,4,11740700,91223,16.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,1.195457e+08,1316.0,21697068.0,6605387.0,23355840.0,13352504.0,23790394.0,19549380.0,11195089.0,160.0,117.0,184.0,113.0,326.0,275.0,141.0,24.0,1807.0,1048.0,759.0,2758606.6,6.0,6.515563e+09
79038,2021,4,11740700,96112,63.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,3.271878e+08,11441.0,45385363.0,29317905.0,34170843.0,45287742.0,62706433.0,71777182.0,38542355.0,1464.0,990.0,1299.0,1658.0,2075.0,2410.0,1545.0,79.0,1807.0,1048.0,759.0,2758606.6,6.0,6.515563e+09
79039,2021,4,11740700,96113,18.000000,0.083269,15.850000,6.0,6.850000,7.89,1.46,14.32,6.670000,1.144000e+07,89.0,1240000.0,0.0,800000.0,0.0,9400000.0,0.0,0.0,20.0,0.0,20.0,0.0,49.0,0.0,0.0,5.0,1807.0,1048.0,759.0,2758606.6,6.0,6.515563e+09


In [36]:
income.to_csv('./income.csv', index=False)